Many researchers work on developing fair classifiers. In this notebook we will train some of these methods and analyze how they perform. 

How might you try to make the classifier we built last time more fair? Would you try to change the data or maybe adjust the predictions?

# Fairness Methods
 
 Methods for achieving algorithmic fairness fall into one of three categories:
 - Pre-processing
 - In-processing
 - Post-processing
 
 ## Pre-processing
 
 Pre-processing methods change the data in order to encourage the algorithm to be more fair. For instance we saw that African-Americans had a higher base rate of recidivism than Caucasians. Maybe if we changed the data so that the base rates looked equal (by for instance reweighing the training data), then the algorithm would learn a more fair classification. 
 
 Pre-processing is method agnostic, but it makes no guarantee about how fair the resulting algorithm will be.
 
 ## In-processing
 
 In-processing methods change the actual learning procedure for the model. Most models are learned via an optimization; in-processing methods often constrain that optimization to a *fair* subspace.
 
 In-processing methods are not method agnostic, so they do not generalize well, but you can get fairness guarantees.
 
 ## Post-processing
 
 Post-processing methods take the predictions of any algorithm and adjust them to be more fair. For instance if our algorithm releases 50% of Caucasians but only 30% of African-Americans, then a most processing method could try to achieve demographic parity by randomly releasing 30% of African-Americans who otherwise would have been detained. How would this achieve demographic parity? 

**Your answer**:

As usual we will import pandas and numpy. We also need to import an optimization package cvxpy. And we will import the fair methods from folders in this directory.

In [4]:
import pandas as pd
import numpy as np
import cvxpy as cvx
from equalized_odds_and_calibration.eq_odds import Model

In [181]:
train = np.loadtxt("train.csv")
test = np.loadtxt("test.csv")
X_pandas = pd.read_csv("X_pandas.csv")
feat_map = list(X_pandas.columns)[1:]
feat_map.append("label")
feat_map.append("prediction")
X_train = train[:,:-2]
y_train = train[:,-2]
X_test = test[:, :-2]
y_test = test[:,-2]

# Reweighing to balance base rates

First we will experiment with a pre-processing method that reweighs our training data to balance the base rate. In our first notebook we calculated the base rates and saw they were different. Now that we have partitioned the data into training and test sets, let's recompute the base rates on the test and train partitions. They should be able the same as before but may be a little different based on the randomization.

In [183]:
recid_rate_cauc = y_train[train[:,feat_map.index('race_Caucasian')]==1].mean()
print("The recidivism base rate for Caucasian defendants is {0:.3f}.".format(recid_rate_cauc))
recid_rate_afr_am = y_train[train[:,feat_map.index('race_African-American')]==1].mean()
print("The recidivism base rate for African-American defendants is {0:.3f}.".format(recid_rate_afr_am))

The recidivism base rate for Caucasian defendants is 0.400.
The recidivism base rate for African-American defendants is 0.518.


In [184]:
perc_afr_am = train[:, feat_map.index('race_African-American')].mean()
perc_cauc = train[:, feat_map.index('race_Caucasian')].mean()
recid_rate_all = y_train.mean()

afr_am_pos_dummy = np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 y_train ==1)

afr_am_neg_dummy = np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 y_train ==0)

cauc_pos_dummy = np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 y_train ==1)

cauc_neg_dummy = np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 y_train ==0)

w_afr_am_pos = perc_afr_am * recid_rate_all \
/ np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 train[:,-2] ==1,).mean()

w_afr_am_neg = perc_afr_am * (1-recid_rate_all) \
/ np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 train[:,-2] ==0,).mean()

w_cauc_pos = perc_cauc * recid_rate_all \
/ np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 train[:,-2] ==1,).mean()

w_cauc_neg = perc_cauc * (1-recid_rate_all) \
/ np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 train[:,-2] ==0,).mean()

w = w_cauc_neg * np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 train[:,-2] ==0,) + \
w_cauc_pos * np.logical_and(train[:,feat_map.index('race_Caucasian')] ==1,\
                 train[:,-2] ==1,) + \
w_afr_am_neg * np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 train[:,-2] ==0,) + \
w_afr_am_pos * np.logical_and(train[:,feat_map.index('race_African-American')] ==1,\
                 train[:,-2] ==1,)

Calculate reweighed base rates.

In [185]:
afr_am_rate_weighed = w[afr_am_pos_dummy].sum()/ \
w[train[:,feat_map.index('race_African-American')]==1].sum()

cauc_rate_weighed = w[cauc_pos_dummy].sum()/ \
w[train[:,feat_map.index('race_Caucasian')]==1].sum()

In [186]:
cauc_rate_weighed

0.4574669187145558

In [187]:
afr_am_rate_weighed

0.4574669187145557

Now use the weights to learn a new logistic regression model

In [171]:
logreg_reweigh = LogisticRegression(C=100000, max_iter=10000, solver = "lbfgs")
logreg_reweigh.fit(X_train,Y, w)

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Can you compute the accuracy and the race-specific false positive and false negative rates (like we did in the last notebook)? *Hint* the FPR for African-Americans is given as an example below

In [176]:
y_pred = logreg_reweigh.predict(test[:, :-2])
y_lab = y_pred.round()
fpr_afr_am_reweigh = y_lab[np.logical_and(test[:, feat_map.index('race_African-American')]==1, \
                                          test[:, -2] == 0)].mean()
print("FPR for African-American after reweighing is {0:.4f}".format(fpr_afr_am_reweigh))

FPR for African-American after reweighing is 0.1786


Does reweighing help achieve equalized odds? Why or why not? 

# Post-processing for Equalized Odds

We will explore a method developed by Moritz Hardt et al (https://arxiv.org/pdf/1610.02413.pdf) that uses post-processing techniques to achieve equalized odds. 

Their methods require a Model data structure that takes two inputs: 1) a matrix of the algorithm predictions and 2) a matrix of the true labels. We have to create such a Model object for each demographic group.

In [11]:
feat_map

['priors_count',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'age',
 'c_charge_degree_F',
 'c_charge_degree_M',
 'race_African-American',
 'race_Caucasian',
 'race_Hispanic',
 'race_Other',
 'sex_Female',
 'sex_Male',
 'label',
 'prediction']

In [20]:
train.shape

(3703, 15)

In [22]:
afr_am_train = train[train[:,feat_map.index("race_African-American")] == 1]
cauc_train = train[train[:,feat_map.index("race_Caucasian")] == 1]
afr_am_test = test[test[:,feat_map.index("race_African-American")] == 1]
cauc_test = test[test[:,feat_map.index("race_Caucasian")] == 1]

In [10]:
feat_map.index("prediction")

14

In [36]:
model_afr_am_train = Model(afr_am_train[:, feat_map.index("prediction")], afr_am_train[:, feat_map.index("label")])
model_cauc_train = Model(cauc_train[:, feat_map.index("prediction")], cauc_train[:, feat_map.index("label")])
model_afr_am_test = Model(afr_am_test[:, feat_map.index("prediction")], afr_am_test[:, feat_map.index("label")])
model_cauc_test = Model(cauc_test[:, feat_map.index("prediction")], cauc_test[:, feat_map.index("label")])

To check that we have done this properly, we can print the models. This shows us the accuracy, FPR, FNR, base rate and mean prediction.

In [25]:
model_afr_am_train

Accuracy:	0.685
F.P. cost:	0.309
F.N. cost:	0.321
Base rate:	0.518
Avg. score:	0.501

In [26]:
model_cauc_train

Accuracy:	0.665
F.P. cost:	0.152
F.N. cost:	0.610
Base rate:	0.400
Avg. score:	0.247

Now let's call the method for post-processing. This method gives us mixing rates that tell us how to randomly resample predicted labels to equalize false positive rates and false negative rates

In [31]:
_, _, mixing_rates = Model.eq_odds(model_afr_am_train, model_cauc_train)

Now we can correct the predictions using these mixing rates

In [32]:
model_afr_am_train_corrected, model_cauc_train_corrected = Model.eq_odds(model_afr_am_train, model_cauc_train, mixing_rates)

In [33]:
model_afr_am_train_corrected

Accuracy:	0.612
F.P. cost:	0.185
F.N. cost:	0.577
Base rate:	0.518
Avg. score:	0.308

In [34]:
model_cauc_train_corrected

Accuracy:	0.649
F.P. cost:	0.193
F.N. cost:	0.587
Base rate:	0.400
Avg. score:	0.281

Did this achieve equalized odds? 

**Your answer**

We should evaluate how the mixing performs on the test partition. Why do we need to do this? Should we get new mixing rates on the test partition or use the mixing rates learned from the train partition?

**Your answer**

In [37]:
model_afr_am_test_corrected, model_cauc_test_corrected = Model.eq_odds(model_afr_am_test, model_cauc_test, mixing_rates)

In [38]:
model_cauc_test_corrected

Accuracy:	0.672
F.P. cost:	0.178
F.N. cost:	0.576
Base rate:	0.378
Avg. score:	0.271

In [39]:
model_afr_am_test_corrected

Accuracy:	0.592
F.P. cost:	0.207
F.N. cost:	0.585
Base rate:	0.531
Avg. score:	0.318

In [40]:
model_cauc_test

Accuracy:	0.691
F.P. cost:	0.136
F.N. cost:	0.594
Base rate:	0.378
Avg. score:	0.238

In [41]:
model_afr_am_test

Accuracy:	0.688
F.P. cost:	0.316
F.N. cost:	0.307
Base rate:	0.531
Avg. score:	0.516

What notion of fairness does this method achieve, if any? What is the effect of this method on the accuracy? Do you think we should use this method for our pre-trial bail application?